<a href="https://colab.research.google.com/github/gjoshi2424/lm-evals/blob/main/Rouge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is used to test using the evaluate package in order to calculate rouge metrics.

In [33]:
!pip install evaluate
!pip install rouge_score

In [34]:
from datasets import load_dataset
import transformers
import torch
import evaluate

In [35]:
#Download dataset from HF
dataset = load_dataset('BI55/MedText', split='train')

In [36]:
sample_data = dataset[0]

In [37]:
#Get model and tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained('gpt2')
model = transformers.AutoModelForCausalLM.from_pretrained('gpt2')

In [38]:
rouge = evaluate.load("rouge")

In [39]:
prompt, completion = sample_data['Prompt'], sample_data['Completion']
print(prompt)
print(completion)

A 50-year-old male presents with a history of recurrent kidney stones and osteopenia. He has been taking high-dose vitamin D supplements due to a previous diagnosis of vitamin D deficiency. Laboratory results reveal hypercalcemia and hypercalciuria. What is the likely diagnosis, and what is the treatment?
This patient's history of recurrent kidney stones, osteopenia, and high-dose vitamin D supplementation, along with laboratory findings of hypercalcemia and hypercalciuria, suggest the possibility of vitamin D toxicity. Excessive intake of vitamin D can cause increased absorption of calcium from the gut, leading to hypercalcemia and hypercalciuria, which can result in kidney stones and bone loss. Treatment would involve stopping the vitamin D supplementation and potentially providing intravenous fluids and loop diuretics to promote the excretion of calcium.


In [40]:
tokenized_prompt = tokenizer(prompt, return_tensors='pt')
with torch.no_grad():
    output = model.generate(**tokenized_prompt, max_length=100, pad_token_id=tokenizer.eos_token_id)
detokenized_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(detokenized_output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A 50-year-old male presents with a history of recurrent kidney stones and osteopenia. He has been taking high-dose vitamin D supplements due to a previous diagnosis of vitamin D deficiency. Laboratory results reveal hypercalcemia and hypercalciuria. What is the likely diagnosis, and what is the treatment?

The diagnosis of vitamin D deficiency is based on a diagnosis of hypercalcemia, hypercalciuria, or hypercalcemia of the kidney. The diagnosis


In [41]:
rouge_results = rouge.compute(predictions=[detokenized_output], references=[completion])
print(rouge_results["rouge1"], rouge_results["rouge2"], rouge_results["rougeL"])

0.45333333333333337 0.20270270270270271 0.30666666666666664


In [45]:
device = torch.device('cpu')
model.to(device)
LIMIT = 10
limited_data_set = dataset.select(range(LIMIT))
predictions = []
references = []
for i in limited_data_set:
  prompt, completion = sample_data['Prompt'], sample_data['Completion']
  tokenized_prompt = tokenizer(prompt, return_tensors='pt')
  with torch.no_grad():
    output = model.generate(**tokenized_prompt, max_length=100, pad_token_id=tokenizer.eos_token_id)
  detokenized_output = tokenizer.decode(output[0], skip_special_tokens=True)
  predictions.append(detokenized_output)
  references.append(completion)
rouge_results = rouge.compute(predictions=predictions, references=references)
print(f"rouge1: {rouge_results["rouge1"]}, rouge2: {rouge_results["rouge2"]}, rougeL: {rouge_results["rougeL"]}")

rouge1: 0.45333333333333325, rouge2: 0.2027027027027027, rougeL: 0.30666666666666664
